reference
https://medium.com/towards-generative-ai/reward-model-training-2209d1befb5f


https://github.com/huggingface/trl/blob/main/examples/scripts/reward_modeling.py

# procedures
1. turn the data into specific format
2. use reward trainer to train the a bert model(input model should be a classify model, whose last layer will use a classify model like sigmoid to get the logits) as reward model

the loss is the mean of (pos logit - neg logit), try to make this number larger
if use center_rewards_coefficient, add a term to the loss, which constrain that the pos and the neg score should have a zero mean, that pos>0 and neg<0

In [ ]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("argilla/comparison-data-falcon-with-feedback")

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

(…)-00000-of-00001-87174140e410b766.parquet:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7401 [00:00<?, ? examples/s]

In [ ]:
#train_df.to_csv('output.csv', index=False)

In [ ]:
#ds['train'][:1]

{'instruction': ['What is Depreciation'],
 'response-1': ["Depreciation is the drop in value of an asset due to wear and tear, age and obsolescence (going out of date) as recorded in an organization's financial records."],
 'response-2': ['What is Depreciation – 10 Important Facts to Know?\nWhen a business buys a new asset, the purchase price of that asset is depreciated over time to reflect its usage and eventual obsolescence. Depreciation expense can be a tax deductible expense and is usually a non-cash expense reported on a company’s income statement and balance sheet. The amount of depreciation expense a company reports each year is the difference between the original purchase price of the asset and what the current value of that asset might be. Here are 10 important facts to know about depreciation:\n1. Depreciation is a non-cash expense. It is an expense that is reported in a business’s income statement and balance sheet and not a cash flow expense.\n2. Depreciation is an account

In [ ]:
import pandas as pd
train_df = pd.DataFrame(ds['train'])

In [ ]:
def mapfunc1(x):
    if x['choose-best']['value'] == [1]:
        return x['response-1']
    elif x['choose-best']['value'] == [2]:
        return x['response-2']
    else:
        return None  # Handle cases where 'choose-best' value is not 1 or 2

def mapfunc2(x):
    if x['choose-best']['value'] == [1]:
        return x['response-2']
    elif x['choose-best']['value'] == [2]:
        return x['response-1']
    else:
        return None  # Handle cases where 'choose-best' value is not 1 or 2

# Apply the mapfunc to each row in the DataFrame
train_df['chosen_response'] = train_df.apply(mapfunc1, axis=1)
train_df['rejected_response'] = train_df.apply(mapfunc2, axis=1)

In [ ]:
#train_df.head()

,instruction,response-1,response-2,choose-best,external_id,chosen_response,rejected_response
0,What is Depreciation,Depreciation is the drop in value of an asset ...,What is Depreciation – 10 Important Facts to K...,"{'user_id': [None], 'value': [1], 'status': ['...",518,Depreciation is the drop in value of an asset ...,What is Depreciation – 10 Important Facts to K...
1,What do you know about the city of Aberdeen in...,"As an AI language model, I don't have personal...",Aberdeen is a city located in the North East o...,"{'user_id': [None], 'value': [2], 'status': ['...",351,Aberdeen is a city located in the North East o...,"As an AI language model, I don't have personal..."
2,Describe thunderstorm season in the United Sta...,Thunderstorm season in the United States and C...,Describe thunderstorm season in the United Sta...,"{'user_id': [None], 'value': [1], 'status': ['...",10567,Thunderstorm season in the United States and C...,Describe thunderstorm season in the United Sta...
3,"When did Peloton IPO?\nOn September 26, 2019, ...",Peloton became a public company via an initial...,When did Peloton IPO?\nPeloton IPO'd on May 26...,"{'user_id': [None], 'value': [1], 'status': ['...",12412,Peloton became a public company via an initial...,When did Peloton IPO?\nPeloton IPO'd on May 26...
4,What is the best way to answer an interview qu...,The first recommended step is to ask clarifyin...,Some of the best ways to answer an interview q...,"{'user_id': [None], 'value': [1], 'status': ['...",2521,The first recommended step is to ask clarifyin...,Some of the best ways to answer an interview q...


In [ ]:
!pip install --upgrade trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments
from trl import RewardTrainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
#Select a base model whch we need to train for reward modeling.
model_name = "distilroberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = examples["instruction"] + "\n" + examples["chosen_response"]

    prompt_plus_rejected_response = examples["instruction"] + "\n" + examples["rejected_response"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }
formatted_dataset = train_df.apply(formatting_func,axis=1)
train_data=formatted_dataset[:5500]
test_data=formatted_dataset[5500:]
#formatted_dataset = formatted_dataset.train_test_split()

In [ ]:
test_data.reset_index(drop=True, inplace=True)

In [ ]:
#pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
from trl import RewardConfig

In [ ]:
# Configuring the training arguments
#training_args = TrainingArguments(
#    output_dir="./reward_model",
#    per_device_train_batch_size=2,
#    evaluation_strategy="steps",
#    logging_steps=1,
#    num_train_epochs = 2,
#    report_to=None,
#)

rd_config = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs=2,
    report_to=None,
    max_length=2048
)

# Loading the RewardTrainer from TRL
trainer = RewardTrainer(
    model=model,
    args=rd_config,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=test_data,
)
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <s>Why is football/soccer such a popular      │ <s>Why is football/soccer such a popular     │ [0.4989, 0.5011] │
│ sport?                                        │ sport?                                       │                  │
│ I believe that football/soccer is a popular   │ Football/soccer has become such a popular    │                  │
│ sport for various reasons. On one hand,       │ sport due to its global appeal, cultural     │                  │
│ everyone has kicked a ball in his or her      │ significance, and the physical and mental    │                  │
│ childhood and can relate to the basic         │ stimulation it provides. It is played and    │                  │
│ movement. On the other hand, it has very      │ watched by people of all ages and            │                  │
│ easy-to-follow rules which makes it possible  │ backgrounds in over 4 billion countries      │                  │
│ to understand the game when you watch it the  │ worldwide, making it one of the most         │                  │
│ first time. Football stars are best known     │ universally loved sports.</s>                │                  │
│ throughout a wide general range of people     │                                              │                  │
│ across the world.</s>                         │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ <s>Best food in the world                     │ <s>Best food in the world                    │ [0.52, 0.48]     │
│ Biriyani</s>                                  │ I love food. I love trying food from         │                  │
│                                               │ different countries. I’ve had some wonderful │                  │
│                                               │ meals in Europe, Asia and Africa, but        │                  │
│                                               │ nothing is quite as good as the food I’ve    │                  │
│                                               │ had here in the US. The thing is, I’ve had   │                  │
│                                               │ some fantastic meals that were prepared by a │                  │
│                                               │ home cook in a house in a suburb of Sydney.  │                  │
│                                               │ I had a fantastic meal in a restaurant in    │                  │
│                                               │ Sydney – and I had to pay $20 for it, which  │                  │
│                                               │ was a bit of a shock to the pocket book.     │                  │
│                                               │ I’ve had some fantastic meals that were      │                  │
│                                               │ prepared by a home cook in a house in a      │                  │
│                                               │ suburb of Sydney.                            │                  │
│                                               │ I love food. I love trying food from         │                  │
│                                               │ different countries. I've had some wonderful │                  │
│                                               │ meals in Europe,...                          │                  │
│                                               │ The thing is, I've had some fantastic        │                  │
│                                               │ meals.

KeyboardInterrupt: 

In [ ]:

dataset = load_dataset('trl-lib/ultrafeedback_binarized')

README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tmp = train_df[:5][['instruction','chosen_response','rejected_response']]

In [ ]:
tmp['input1']=tmp['instruction'] + "\n" +tmp['chosen_response']
tmp['input2']=tmp['instruction'] + "\n" +tmp['rejected_response']

In [ ]:
#input_text = "What is Depreciation" + "\n" + "Depreciation is the drop in value of an asset due to wear and tear, age and obsolescence (going out of date) as recorded in an organization's financial records."

for i in tmp['input1']:
  inputs = tokenizer(i, return_tensors="pt")

  # Evaluate the input data
  outputs = model(**inputs)
  # reward_score = outputs.logits
  print(outputs.logits)

tensor([[0.0270]], grad_fn=<AddmmBackward0>)
tensor([[0.0242]], grad_fn=<AddmmBackward0>)
tensor([[0.0174]], grad_fn=<AddmmBackward0>)
tensor([[0.0302]], grad_fn=<AddmmBackward0>)
tensor([[0.0245]], grad_fn=<AddmmBackward0>)


In [ ]:

for i in tmp['input2']:
  inputs = tokenizer(i, return_tensors="pt")

  # Evaluate the input data
  outputs = model(**inputs)
  # reward_score = outputs.logits
  print(outputs.logits)

tensor([[0.0201]], grad_fn=<AddmmBackward0>)
tensor([[0.0389]], grad_fn=<AddmmBackward0>)
tensor([[0.0142]], grad_fn=<AddmmBackward0>)
tensor([[0.0286]], grad_fn=<AddmmBackward0>)
tensor([[0.0331]], grad_fn=<AddmmBackward0>)


In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.3 MB/s eta 0:00:00


In [ ]:
import warnings

import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, HfArgumentParser

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)
from trl.commands.cli_utils import RewardScriptArguments


In [ ]:
"""
Full training:
python examples/scripts/reward_modeling.py \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir Qwen2-0.5B-Reward \
    --per_device_train_batch_size 8 \
    --num_train_epochs 1 \
    --gradient_checkpointing True \
    --learning_rate 1.0e-5 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 2048

LoRA:
python examples/scripts/reward_modeling.py \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir Qwen2-0.5B-Reward-LoRA \
    --per_device_train_batch_size 8 \
    --num_train_epochs 1 \
    --gradient_checkpointing True \
    --learning_rate 1.0e-4 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 2048 \
    --use_peft \
    --lora_r 32 \
    --lora_alpha 16
"""


'\nFull training:\npython examples/scripts/reward_modeling.py     --model_name_or_path Qwen/Qwen2-0.5B-Instruct     --dataset_name trl-lib/ultrafeedback_binarized     --output_dir Qwen2-0.5B-Reward     --per_device_train_batch_size 8     --num_train_epochs 1     --gradient_checkpointing True     --learning_rate 1.0e-5     --logging_steps 25     --eval_strategy steps     --eval_steps 50     --max_length 2048\n\nLoRA:\npython examples/scripts/reward_modeling.py     --model_name_or_path Qwen/Qwen2-0.5B-Instruct     --dataset_name trl-lib/ultrafeedback_binarized     --output_dir Qwen2-0.5B-Reward-LoRA     --per_device_train_batch_size 8     --num_train_epochs 1     --gradient_checkpointing True     --learning_rate 1.0e-4     --logging_steps 25     --eval_strategy steps     --eval_steps 50     --max_length 2048     --use_peft     --lora_r 32     --lora_alpha 16\n'

In [ ]:
"""
Full training:
python examples/scripts/reward_modeling.py \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir Qwen2-0.5B-Reward \
    --per_device_train_batch_size 8 \
    --num_train_epochs 1 \
    --gradient_checkpointing True \
    --learning_rate 1.0e-5 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 2048

LoRA:
python examples/scripts/reward_modeling.py \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --dataset_name trl-lib/ultrafeedback_binarized \
    --output_dir Qwen2-0.5B-Reward-LoRA \
    --per_device_train_batch_size 8 \
    --num_train_epochs 1 \
    --gradient_checkpointing True \
    --learning_rate 1.0e-4 \
    --logging_steps 25 \
    --eval_strategy steps \
    --eval_steps 50 \
    --max_length 2048 \
    --use_peft \
    --lora_r 32 \
    --lora_alpha 16
"""


if __name__ == "__main__":
    parser = HfArgumentParser((RewardScriptArguments, RewardConfig, ModelConfig))
    script_args, training_args, model_config = parser.parse_args_into_dataclasses()
    training_args.gradient_checkpointing_kwargs = dict(use_reentrant=False)

    ################
    # Model & Tokenizer
    ################
    torch_dtype = (
        model_config.torch_dtype
        if model_config.torch_dtype in ["auto", None]
        else getattr(torch, model_config.torch_dtype)
    )
    quantization_config = get_quantization_config(model_config)
    model_kwargs = dict(
        revision=model_config.model_revision,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
        use_cache=False if training_args.gradient_checkpointing else True,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_config.model_name_or_path, trust_remote_code=model_config.trust_remote_code, use_fast=True
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_config.model_name_or_path, num_labels=1, trust_remote_code=model_config.trust_remote_code, **model_kwargs
    )
    # Align padding tokens between tokenizer and model
    model.config.pad_token_id = tokenizer.pad_token_id

    # If post-training a base model, use ChatML as the default template
    if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)

    if model_config.use_peft and model_config.lora_task_type != "SEQ_CLS":
        warnings.warn(
            "You are using a `task_type` that is different than `SEQ_CLS` for PEFT. This will lead to silent bugs"
            " Make sure to pass --lora_task_type SEQ_CLS when using this script with PEFT."
        )

usage: colab_kernel_launcher.py [-h] [--dataset_name DATASET_NAME]
                                [--dataset_train_split DATASET_TRAIN_SPLIT]
                                [--dataset_test_split DATASET_TEST_SPLIT] [--config CONFIG]
                                [--gradient_checkpointing_use_reentrant [GRADIENT_CHECKPOINTING_USE_REENTRANT]]
                                --output_dir OUTPUT_DIR
                                [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                                [--do_train [DO_TRAIN]] [--do_eval [DO_EVAL]]
                                [--do_predict [DO_PREDICT]] [--eval_strategy {no,steps,epoch}]
                                [--prediction_loss_only [PREDICTION_LOSS_ONLY]]
                                [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
                                [--per_device_eval_batch_size PER_DEVICE_EVAL_BATCH_SIZE]
                                [--per_gpu_train_batch_size PER_GPU_TRAIN_BATCH_SI

SystemExit: 2

In [ ]:
 dataset = load_dataset(script_args.dataset_name)

NameError: name 'script_args' is not defined